# Notebook for Generating Audios from Make-An-Audio 2 Model

In [1]:
%cd Make-An-Audio-2
!pwd

/home/ec2-user/Make-An-Audio-2
/home/ec2-user/Make-An-Audio-2


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Obtaining BigVGAN from git+https://github.com/awei-6/BigVGAN.git@main#egg=BigVGAN (from -r requirements.txt (line 26))
  Updating ./src/bigvgan clone (to revision main)
  Running command git fetch -q --tags
  Running command git reset --hard -q ac17a79ca9abe1a49a7c23f52f73d405fbd90da4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 958.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 121.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 129.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 117.5 MB/s eta 0:00:0000:0100:01

In [2]:
# Used to calculated CLAP scores
!python -m wav_evaluation.cal_clap_score --csv_path maa1_ft.tsv

/opt/conda/envs/pytorch/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly i

In [2]:
# Used to produce spectrograms
!python -m preprocess.mel_spec --tsv_path make_spec.tsv --num_gpus 1 --max_duration 10

3it [00:00, 62291.64it/s]
100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 28.16it/s]
[]
3it [00:00, 81180.08it/s]
/opt/conda/envs/pytorch/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|█████████████████████████████████████████████| 3/3 [00:02<00:00,  1.11it/s]
proceoss mel done
done


In [ ]:
# Used to produce GPT-3.5 structured captions
!python preprocess/n2s_by_openai.py --tsv_path maa2.tsv

In [ ]:
# This block is to generate audios from captions in test_df.csv

import pandas as pd
import subprocess 

df = pd.read_csv('test_df.csv')
for _, row in df.iterrows():
    name = row['Audio_mp3'][:-4]
    cap = row['Caption']
    str_cap = row['Structured_Caption']

    command = f"python -m scripts.gen_wav --scale 4  --duration 10 --save_name gen_wav/{name} --prompt \"{cap}\" --struct_prompt \"{str_cap}\""
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    print(result.stderr)

In [ ]:
# This block is to finish off generating captions (I had to stop halfway since I ran out of colab time. I saved the files I'd already generated in progress.pkl)

import pandas as pd
import subprocess
import pickle

file_path = 'progress.pkl'

# Open the pickle file in read-binary mode and load the dictionary
with open(file_path, 'rb') as file:
    comp = pickle.load(file)

df = pd.read_csv('maa2_struct.tsv', delimiter='\t')
for _, row in df.iterrows():
    name = row['name'][:-4]
    if name in comp:
        continue
    cap = row['caption']
    str_cap = row['struct_cap']

    command = f"python -m scripts.gen_wav --scale 4  --duration 10 --save_name gen_wav/{name} --prompt \"{cap}\" --struct_prompt \"{str_cap}\""
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    print(result.stderr)